# Компьютерное зрение. Базовый курс

## Аугментация данных

#### Условие

**Дано:** выборка идеально растеризованных страниц документов.

**Необходимо:**

* Сгенерировать «фотореалистичную» выборку изображений, имитирующих фотографии страниц этих документов, лежащих на произвольных поверхностях. Размер получаемой в итоге выборки должен в 10 раз превышать исходную.

* Прислать исходный код алгоритма аугментации и ссылку на полученную в результате выборку. Скорость работы алгоритма не оценивается. Срок – до 21 марта.

Рекомендуется применять к изображениям различные преобразования, ставящие своей целью сымитировать физический процесс регистрации страницы документа камерой, такие как:

* проективное преобразование,

* добавление текстур поверхности и бумаги

* дефектов печати,

* шумов (и шумового фильтра!),

* освещения (бликов, теней, ...),

* эффекта глубины резкости,

т.е. всего, что добавит схожести с реальными фотографиями этих документов (полезно вспомнить материал лекции №2).

Разрешается использовать внешние библиотеки для реализации отдельных этапов аугментаций. Готовые программы для 3D моделирования использовать нельзя!

#### Решение

In [1]:
IMAGES_NUMBER = 10
SAMPLES_NUMBER = 10
MIN_PERSPECTIVE_DEFLECTION = 0
MAX_PERSPECTIVE_DEFLECTION = 200
DEFLECTION_NUMBER = 8
NUM_CHANNELS = 3
MIN_COMPONENT_VALUE = 0
MAX_COMPONENT_VALUE = 255

In [2]:
import cv2
import numpy as np

In [3]:
def viewImage(image, name_of_window):
    cv2.namedWindow(name_of_window, cv2.WINDOW_NORMAL)
    cv2.imshow(name_of_window, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Для создания наиболее правдоподобного эффекта регистрации страниц документов камерой применим несколько преобразований над исходными изображениями.

1. Текстура мятой бумаги

На каждое изображение наложим одно (выбранное случайным образом из двух предложенных) изображение мятой бумаги.

In [4]:
crumpled = [cv2.imread("Crumpled" + str(number) + ".png", cv2.IMREAD_COLOR) for number in range(2)]

In [5]:
def make_crumpled(image):
    height, width = image.shape[:2]
    paper = cv2.resize(crumpled[np.random.randint(2)], (width, height))
    result = 0.4 * paper + 0.6 * image
    return result

2. Освещение

Сымитируем 4 возможных варианта освещения документа с помощью радиального градиента. Между собой упорядочим их в порядке возрастания эффекта: первый вариант - отсутствие освещения, четвертый вариант - ярко выраженное светлое пятно (как от вспышки).

In [6]:
def generate_highlight(intensity):
    curr_highlight = np.array([30 + intensity*3])
    for i in range(1500):
        new_highlight = np.full((2*i+3, 2*i+3), 29 + intensity*3 - i // (20 - intensity))
        new_highlight[1:-1, 1:-1] = curr_highlight
        curr_highlight = new_highlight
    return curr_highlight

In [7]:
highlights = [np.full((3001, 3001), 0)]
for intensity in range(13, 16):
    highlights.append(generate_highlight(intensity))

In [8]:
def add_highlight(image, highlights):
    highlight = highlights[np.random.randint(0, 4)]
    height = image.shape[0]
    width = image.shape[1]
    highlight_size = highlight.shape[0]
    new_image = image.astype(np.int32).transpose((2, 0, 1))
    new_first = highlight.copy()
    new_first[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)] += new_image[0]
    new_second = highlight.copy()
    new_second[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)] += new_image[1]
    new_third = highlight.copy()
    new_third[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)] += new_image[2]
    new_image[0] = new_first[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)].clip(MIN_COMPONENT_VALUE, MAX_COMPONENT_VALUE)
    new_image[1] = new_second[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)].clip(MIN_COMPONENT_VALUE, MAX_COMPONENT_VALUE)
    new_image[2] = new_third[((highlight_size - height) // 2):((highlight_size - height) // 2 + height), ((highlight_size - width) // 2):((highlight_size - width) // 2 + width)].clip(MIN_COMPONENT_VALUE, MAX_COMPONENT_VALUE)
    new_image = new_image.astype('uint8').transpose((1, 2, 0))
    return new_image

3. Имитация трясущейся камеры

Используем размытие по Гауссу со случайно заданными параметрами $\sigma_x$ и $\sigma_y$ (берутся из равномерного распределения на отрезке $[0, 4]$).

In [9]:
def add_blur(image):
    return cv2.GaussianBlur(src=image, ksize=(0, 0), sigmaX=np.random.rand()*4, sigmaY=np.random.rand()*4)

4. Тень

Каждому "листу бумаги" для придания объема добавим заранее заготовленную тень.

In [10]:
shadow = cv2.imread('Shadow.png', cv2.IMREAD_COLOR)

In [11]:
def add_shadow(image):
    height, width = image.shape[:2]
    curr_shadow = cv2.resize(shadow, (width + 40, height + 40))
    curr_shadow[20:-20, 20:-20, :] = image
    return curr_shadow

5. Перспектива

Наконец, для создания эффектов "документ лежит на столе" и "документ висит на стене" воспользуемся преобразованиями перспективы.

In [12]:
def add_perspective(image):
    height, width = image.shape[:2]
    pts1 = np.float32([[0, 0], [width, 0],
                       [0, height], [width, height]])
    deflections = np.random.randint(MIN_PERSPECTIVE_DEFLECTION, MAX_PERSPECTIVE_DEFLECTION, DEFLECTION_NUMBER) * np.random.randint(2, size=8)
    deflections[0], deflections[1] = max(deflections[0], deflections[1]), min(deflections[0], deflections[1])
    pts2 = np.float32([[deflections[0], deflections[2]], [width - deflections[0], deflections[3]],
                       [deflections[1], height - deflections[2]], [width - deflections[1], height - deflections[3]]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result = cv2.warpPerspective(image, matrix, (width, height), borderValue=(MAX_COMPONENT_VALUE, MAX_COMPONENT_VALUE, MAX_COMPONENT_VALUE))
    return result

Итоговая функция, искажающая изображение и создающая эффект бумажного документа:

In [13]:
def distort_image(image, image_number, number):
    for i in range(number):
        crumpled_image = make_crumpled(image)
        image_with_highlight = add_highlight(crumpled_image, highlights)
        image_with_blur = add_blur(image_with_highlight)
        image_with_shadow = add_shadow(image_with_blur)
        result = add_perspective(image_with_shadow)
        cv2.imwrite('./dataset/' + str(image_number) + '_' + str(i) + '.png', result)

In [14]:
for i in range(IMAGES_NUMBER):
    image = cv2.imread('./sources/' + str(i + 1) + '.png', cv2.IMREAD_COLOR)
    distort_image(image, i + 1, SAMPLES_NUMBER)